In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

## 1 year / 1 league

In [3]:
pretag = "https://www.worldfootball.net"

In [4]:
year = 2019
champion = pd.read_html("https://www.worldfootball.net/schedule/ita-serie-a-{}-{}-spieltag/33/".format(year, year+1))[3]["Team.1"][0]
page = requests.get("https://www.worldfootball.net/players/ita-serie-a-{}-{}/".format(year, year+1))
soup = BeautifulSoup(page.content, 'html.parser')

**the table is listed in "td"s - td_length used to loop through them**

In [5]:
td_length = len(soup.findAll('td'))

**finding the teamname and their link to the squadlist - the link for the squads are 4 places away from the team name**

In [6]:
print(soup.findAll('td')[84].text)
print(soup.findAll('td')[88].a['href'])

Juventus
/teams/juventus/2020/2/


In [7]:
for i in range(td_length):
    if soup.findAll('td')[i].text == champion:
        x = champion
        print(x)
        y = soup.findAll('td')[i+4].a['href']
        print(y)

Juventus
/teams/juventus/2020/2/


In [8]:
squad_list = pd.read_html(pretag+y)[1]

## 1 year / Data cleaning

**Removing all non-numeric rown, since only players need to counted and only they have kit numbers**

In [9]:
# Removing all non-numeric rown, since only players need to counted and only they have kit numbers
squad_list['Goalkeeper.1'] = pd.to_numeric(squad_list['Goalkeeper.1'], errors='coerce')
squad_list = squad_list[squad_list['Goalkeeper.1'].notna()]
# Counting the nationalities and adding year column
squad_list_counted2 = squad_list["Goalkeeper.4"].value_counts().to_frame()
squad_list_counted2['Year'] = year+1
# Renaming
squad_list_counted2.columns = ['Count', 'Year']

## Multiple Years

**Creating a dictionary with the country and league names**

In [10]:
country_dict = {
    "Albania" : "alb-kategoria-superiore",
    "Andorra" : "and-1a-divisio",
    "Armenia" : "arm-premier-league", 
    "Austria" : "aut-bundesliga", 
    "Azerbaijan" : "aze-premyer-liqasi",
    "Belarus" : "blr-cempionat", #single-year league
    "Belgium" : "bel-eerste-klasse",
    "Bosnia-Herzegovina" : "bih-premier-liga",
    "Bulgaria" : "bul-a-grupa",
    "Croatia" : "cro-1-hnl",
    "Cyprus" : "cyp-first-division",
    "Czech Republic" : "cze-1-fotbalova-liga",
    "Denmark" : "den-superliga",
    "England" : "eng-premier-league",
    "Estonia" : "/est-meistriliiga", #single-year league
    "Faroe Islands" : "fro-effodeildin", #single-year league
    "Finland" : "fin-veikkausliiga/" ,#single-year league
    "France" : "fra-ligue-1",
#     "Georgia" : "geo-umaglesi-liga", #single year league, error
    "Germany" : "bundesliga",
    "Gibraltar" : "gib-premier-divison",
    "Greece" : "gre-superleague", #changed names last year
    "Hungary" : "hun-nb-i",
    "Iceland" : "isl-urvalsdeild", #single-year league
    "Israel" : "isr-ligat-haal",
    "Italy" : "ita-serie-a",
    "Kazakhstan" : "kaz-premier-liga", #sinle-year league
    "Kosovo" : "kos-superliga",
    "Latvia" : "lat-virsliga", #single-year league
    "Lithuania" : "ltu-a-lyga", #single-year league
    "Luxembourg" : "lux-nationaldivisioun",
#     "Malta" : "mlt-premier-league", error
#     "Moldova" : "mda-divizia-nationala", error
    "Montenegro" : "mne-1-cfl",
    "Netherlands" : "ned-eredivisie",
    "North Macedonia" : "mkd-1-mfl",
    "Norway" : "nor-eliteserien",
    "Northern Ireland" : "nir-premier-league",
    "Poland" : "pol-ekstraklasa",
    "Portugal" : "por-primeira-liga",
    "Ireland" : "irl-premier-division",
    "Romania" : "rou-liga-1",
    "Russia" : "rus-premier-liga",
    "Scotland" : "sco-premiership",
    "Serbia" : "srb-super-liga",
    "Slovakia" : "svk-super-liga",
    "Slovenia" : "svn-1-snl",
    "Spain" : "esp-primera-division",
    "Sweden" : "swe-allsvenskan", #single-year
    "Switzerland" : "sui-super-league",
    "Turkey" : "tur-sueperlig",
    "Ukraine" : "ukr-premyer-liga",
    "Wales" : "wal-premier-league"  
}

In [11]:
dict_length = len(country_dict)

In [12]:
squad_list_counted = pd.DataFrame()

## Multiple Years / Looping through all the years and leagues

In [13]:
for country_index in range(len(country_dict)):
    for year in range(2016, 2021):
        
        try: # single-year leagues
            page2 = requests.get("https://www.worldfootball.net/schedule/{}-{}-spieltag/1/".format(country_dict[list(country_dict.keys())[country_index]],year))
            soup2 = BeautifulSoup(page2.content, 'html.parser')
            spieltag = soup2.findAll('option')[-1].text.split(".")[0]
            champion = pd.read_html("https://www.worldfootball.net/schedule/{}-{}-spieltag/{}/".format(country_dict[list(country_dict.keys())[country_index]],year,spieltag))[3]["Team.1"][0]
            page = requests.get("https://www.worldfootball.net/players/{}-{}/".format(country_dict[list(country_dict.keys())[country_index]],year))
        except: # double-year leagues
            try:
                page2 = requests.get("https://www.worldfootball.net/schedule/{}-{}-{}-spieltag/1/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1))
                soup2 = BeautifulSoup(page2.content, 'html.parser')
                spieltag = soup2.findAll('option')[-1].text.split(".")[0]
                champion = pd.read_html("https://www.worldfootball.net/schedule/{}-{}-{}-spieltag/{}/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1, spieltag))[3]["Team.1"][0]
                page = requests.get("https://www.worldfootball.net/players/{}-{}-{}/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1))
            except:
                if(list(country_dict.keys())[country_index] == "Spain") :
                    page2 = requests.get("https://www.worldfootball.net/schedule/{}-{}-{}-spieltag_2/1/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1))
                    soup2 = BeautifulSoup(page2.content, 'html.parser')
                    spieltag = soup2.findAll('option')[-1].text.split(".")[0]
                    champion = pd.read_html("https://www.worldfootball.net/schedule/{}-{}-{}-spieltag_2/{}/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1, spieltag))[3]["Team.1"][0]
                    page = requests.get("https://www.worldfootball.net/players/{}-{}-{}_2/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1))
                elif(list(country_dict.keys())[country_index] == "Belgium") :
                    page2 = requests.get("https://www.worldfootball.net/schedule/{}-a-{}-{}-spieltag/1/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1))
                    soup2 = BeautifulSoup(page2.content, 'html.parser')
                    spieltag = soup2.findAll('option')[-1].text.split(".")[0]
                    champion = pd.read_html("https://www.worldfootball.net/schedule/{}-a-{}-{}-spieltag/{}/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1, spieltag))[3]["Team.1"][0]
                    page = requests.get("https://www.worldfootball.net/players/{}-a-{}-{}/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1))
                elif (list(country_dict.keys())[country_index] == "Bulgaria") :
                    page2 = requests.get("https://www.worldfootball.net/schedule/bul-parva-liga-{}-{}-spieltag/1/".format(year, year+1))
                    soup2 = BeautifulSoup(page2.content, 'html.parser')
                    spieltag = soup2.findAll('option')[-1].text.split(".")[0]
                    champion = pd.read_html("https://www.worldfootball.net/schedule/bul-parva-liga-{}-{}-spieltag/{}/".format(year, year+1, spieltag))[3]["Team.1"][0]
                    page = requests.get("https://www.worldfootball.net/players/bul-parva-liga-{}-{}/".format(year, year+1))                    
                elif (list(country_dict.keys())[country_index] == "Greece") :
                    page2 = requests.get("https://www.worldfootball.net/schedule/gre-super-league-{}-{}-spieltag/1/".format(year, year+1))
                    soup2 = BeautifulSoup(page2.content, 'html.parser')
                    spieltag = soup2.findAll('option')[-1].text.split(".")[0]
                    champion = pd.read_html("https://www.worldfootball.net/schedule/gre-super-league-{}-{}-spieltag/{}/".format(year, year+1, spieltag))[3]["Team.1"][0]
                    page = requests.get("https://www.worldfootball.net/players/gre-super-league-{}-{}/".format(year, year+1))                    
                elif (list(country_dict.keys())[country_index] == "Luxembourg"):
                    page2 = requests.get("https://www.worldfootball.net/schedule/lux-nationaldivision-{}-{}-spieltag/1/".format(year, year+1))
                    soup2 = BeautifulSoup(page2.content, 'html.parser')
                    spieltag = soup2.findAll('option')[-1].text.split(".")[0]
                    champion = pd.read_html("https://www.worldfootball.net/schedule/lux-nationaldivision-{}-{}-spieltag/{}/".format(year, year+1, spieltag))[3]["Team.1"][0]
                    page = requests.get("https://www.worldfootball.net/players/lux-nationaldivision-{}-{}/".format(year, year+1))
                elif(list(country_dict.keys())[country_index] == "Slovenia"):
                    page2 = requests.get("https://www.worldfootball.net/schedule/svn-prvaliga-{}-{}-spieltag/1/".format(year, year+1))
                    soup2 = BeautifulSoup(page2.content, 'html.parser')
                    spieltag = soup2.findAll('option')[-1].text.split(".")[0]
                    champion = pd.read_html("https://www.worldfootball.net/schedule/svn-prvaliga-{}-{}-spieltag/{}/".format(year, year+1, spieltag))[3]["Team.1"][0]
                    page = requests.get("https://www.worldfootball.net/players/svn-prvaliga-{}-{}/".format(year, year+1))                   
                    
#         page = requests.get("https://www.worldfootball.net/players/{}-{}-{}/".format(country_dict[list(country_dict.keys())[country_index]],year, year+1))
        soup = BeautifulSoup(page.content, 'html.parser')
        td_length = len(soup.findAll('td'))

            
        for i in range(td_length):
            if soup.findAll('td')[i].text == champion:
                x = champion
                y = soup.findAll('td')[i+4].a['href']

        print(list(country_dict.keys())[country_index])
        squad_list = pd.read_html(pretag+y)[1]
        squad_list['Goalkeeper.1'] = pd.to_numeric(squad_list['Goalkeeper.1'], errors='coerce')
        squad_list = squad_list[squad_list['Goalkeeper.1'].notna()]
        squad_list_counted2 = squad_list["Goalkeeper.4"].value_counts().to_frame()
        squad_list_counted2['Year'] = year+1
        squad_list_counted2['Country'] = list(country_dict.keys())[country_index]
        squad_list_counted2.reset_index(inplace=True)
        squad_list_counted2.columns = ['Origin','Count', 'Year', 'Country']
        squad_list_counted = pd.concat([squad_list_counted, squad_list_counted2])

Albania
Albania
Albania
Albania
Albania
Andorra
Andorra
Andorra
Andorra
Andorra
Armenia
Armenia
Armenia
Armenia
Armenia
Austria
Austria
Austria
Austria
Austria
Azerbaijan
Azerbaijan
Azerbaijan
Azerbaijan
Azerbaijan
Belarus
Belarus
Belarus
Belarus
Belarus
Belgium
Belgium
Belgium
Belgium
Belgium
Bosnia-Herzegovina
Bosnia-Herzegovina
Bosnia-Herzegovina
Bosnia-Herzegovina
Bosnia-Herzegovina
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Croatia
Croatia
Croatia
Croatia
Croatia
Cyprus
Cyprus
Cyprus
Cyprus
Cyprus
Czech Republic
Czech Republic
Czech Republic
Czech Republic
Czech Republic
Denmark
Denmark
Denmark
Denmark
Denmark
England
England
England
England
England
Estonia
Estonia
Estonia
Estonia
Estonia
Faroe Islands
Faroe Islands
Faroe Islands
Faroe Islands
Faroe Islands
Finland
Finland
Finland
Finland
Finland
France
France
France
France
France
Germany
Germany
Germany
Germany
Germany
Gibraltar
Gibraltar
Gibraltar
Gibraltar
Gibraltar
Greece
Greece
Greece
Greece
Greece
Hungary
Hungary
Hungary
H

In [14]:
squad_list_counted

,Origin,Count,Year,Country
0,Albania,17,2017,Albania
1,Brazil,5,2017,Albania
2,North Macedonia,2,2017,Albania
3,Croatia,2,2017,Albania
4,Greece,1,2017,Albania
...,...,...,...,...
5,Northern Ireland,1,2020,Wales
0,England,11,2021,Wales
1,Wales,7,2021,Wales
2,Ireland,1,2021,Wales


In [15]:
squad_list_counted2 = squad_list_counted

In [16]:
squad_list_counted2['Count_Origin'] = np.where (squad_list_counted2['Country'] == squad_list_counted2['Origin'], squad_list_counted2['Count'], 0)

In [17]:
squad_list_counted3 = squad_list_counted2.groupby(['Country','Year'], as_index=False).sum()

In [18]:
squad_list_counted3

,Country,Year,Count,Count_Origin
0,Albania,2017,28,17
1,Albania,2018,25,16
2,Albania,2019,23,17
3,Albania,2020,19,15
4,Albania,2021,25,19
...,...,...,...,...
245,Wales,2017,24,9
246,Wales,2018,22,6
247,Wales,2019,35,33
248,Wales,2020,22,8


In [19]:
squad_list_counted3['Ratio'] = np.round(squad_list_counted3['Count_Origin']/squad_list_counted3['Count']*100,2)

In [20]:
squad_list_counted4 = squad_list_counted2.groupby(['Country'], as_index=False).sum()

In [21]:
squad_list_counted4.drop('Year', axis=1, inplace=True)

In [22]:
squad_list_counted4['Ratio'] = np.round(squad_list_counted4['Count_Origin']/squad_list_counted4['Count']*100,2)

**Once both dataframes are ready they get exported**

In [23]:
squad_list_counted3.to_csv('yearly_breakdown.csv', encoding='utf-8', index=False)
squad_list_counted4.to_csv('5_year_agg.csv', encoding='utf-8', index=False)